In [15]:
import sys
import os
import pandas as pd
from pathlib import Path
import numpy as np
import time
import requests
from IPython.display import display
from dotenv import load_dotenv

# Cấu hình đường dẫn để import từ 'src' 
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))

# IMPORT CÁC HÀM TÙY CHỈNH
try:
    from cleaning_diem_thi import run_full_preprocessing, display_stats_report
    print(" Import các hàm từ 'src' thành công!")
except ImportError as e:
    print(f" LỖI KHÔNG TÌM THẤY MODULE: {e}")
try: 
    from preprocessing_diem_thi import process_files_vectorized, crawl_province, geocode_province
    print(" Import thành công")
except ImportError as e: 
    print(f" LỖI KHÔNG TÌM THẤY MODULE: {e}")

# Cấu hình đường dẫn Data và Config 
DATA_DIR = project_root / 'data'
CONFIG_PATH = DATA_DIR / 'to_hop.json' 
load_dotenv()
MAPBOX_TOKEN = os.getenv('MAPBOX_TOKEN')
# Định nghĩa các tham số
years_to_process = {
    2019: CONFIG_PATH,
    2020: CONFIG_PATH,
    2021: CONFIG_PATH,
    2022: CONFIG_PATH,
    2023: CONFIG_PATH,
    2024: CONFIG_PATH,
}

print(f"Thiết lập thành công. Sẵn sàng xử lý {len(years_to_process)} năm.")

 Import các hàm từ 'src' thành công!
 Import thành công
Thiết lập thành công. Sẵn sàng xử lý 6 năm.


In [7]:
NGUONG_DIEM_LIET = 1.0
NGUONG_DIEM_SAN_TO_HOP = 15.0
all_stats_data = run_full_preprocessing(
    years_to_process, 
    DATA_DIR, 
    nguong_diem_liet=NGUONG_DIEM_LIET, 
    nguong_diem_to_hop=NGUONG_DIEM_SAN_TO_HOP
)

In [9]:
# Báo cáo thông số
display_stats_report(all_stats_data)

In [4]:
STEP = 0.05
process_files_vectorized(NGUONG_DIEM_SAN_TO_HOP, STEP)


In [16]:
# Lấy kinh độ, vĩ độ cho từng tỉnh thành
provinces = [
    {'code': '01', 'name': 'Hà Nội'},
    {'code': '02', 'name': 'Thành phố Hồ Chí Minh'},
    {'code': '03', 'name': 'Hải Phòng'},
    {'code': '04', 'name': 'Đà Nẵng'},
    {'code': '05', 'name': 'Hà Giang'},
    {'code': '06', 'name': 'Cao Bằng'},
    {'code': '07', 'name': 'Lai Châu'},
    {'code': '08', 'name': 'Lào Cai'},
    {'code': '09', 'name': 'Tuyên Quang'},
    {'code': '10', 'name': 'Lạng Sơn'},
    {'code': '11', 'name': 'Bắc Kạn'},
    {'code': '12', 'name': 'Thái Nguyên'},
    {'code': '13', 'name': 'Yên Bái'},
    {'code': '14', 'name': 'Sơn La'},
    {'code': '15', 'name': 'Phú Thọ'},
    {'code': '16', 'name': 'Vĩnh Phúc'},
    {'code': '17', 'name': 'Quảng Ninh'},
    {'code': '18', 'name': 'Bắc Giang'},
    {'code': '19', 'name': 'Bắc Ninh'},
    {'code': '21', 'name': 'Hải Dương'},
    {'code': '22', 'name': 'Hưng Yên'},
    {'code': '23', 'name': 'Hòa Bình'},
    {'code': '24', 'name': 'Hà Nam'},
    {'code': '25', 'name': 'Nam Định'},
    {'code': '26', 'name': 'Thái Bình'},
    {'code': '27', 'name': 'Ninh Bình'},
    {'code': '28', 'name': 'Thanh Hóa'},
    {'code': '29', 'name': 'Nghệ An'},
    {'code': '30', 'name': 'Hà Tĩnh'},
    {'code': '31', 'name': 'Quảng Bình'},
    {'code': '32', 'name': 'Quảng Trị'},
    {'code': '33', 'name': 'Thừa Thiên Huế'},
    {'code': '34', 'name': 'Quảng Nam'},
    {'code': '35', 'name': 'Quảng Ngãi'},
    {'code': '36', 'name': 'Kon Tum'},
    {'code': '37', 'name': 'Bình Định'},
    {'code': '38', 'name': 'Gia Lai'},
    {'code': '39', 'name': 'Phú Yên'},
    {'code': '40', 'name': 'Đắk Lắk'},
    {'code': '41', 'name': 'Khánh Hòa'},
    {'code': '42', 'name': 'Lâm Đồng'},
    {'code': '43', 'name': 'Bình Phước'},
    {'code': '44', 'name': 'Bình Dương'},
    {'code': '45', 'name': 'Ninh Thuận'},
    {'code': '46', 'name': 'Tây Ninh'},
    {'code': '47', 'name': 'Bình Thuận'},
    {'code': '48', 'name': 'Đồng Nai'},
    {'code': '49', 'name': 'Long An'},
    {'code': '50', 'name': 'Đồng Tháp'},
    {'code': '51', 'name': 'An Giang'},
    {'code': '52', 'name': 'Bà Rịa-Vũng Tàu'},
    {'code': '53', 'name': 'Tiền Giang'},
    {'code': '54', 'name': 'Kiên Giang'},
    {'code': '55', 'name': 'Cần Thơ'},
    {'code': '56', 'name': 'Bến Tre'},
    {'code': '57', 'name': 'Vĩnh Long'},
    {'code': '58', 'name': 'Trà Vinh'},
    {'code': '59', 'name': 'Sóc Trăng'},
    {'code': '60', 'name': 'Bạc Liêu'},
    {'code': '61', 'name': 'Cà Mau'},
    {'code': '62', 'name': 'Điện Biên'},
    {'code': '63', 'name': 'Đắk Nông'},
    {'code': '64', 'name': 'Hậu Giang'}
]
print(MAPBOX_TOKEN)
geocode_province(provinces, MAPBOX_TOKEN)
crawl_province()

In [18]:
from pathlib import Path
import sys

PROJECT_ROOT = Path("/home/truong/Documents/introduction-to-data-science")

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
try: 
    from chuan_hoa_diem_chuan import process_diem_chuan
    print('Import thành công')
except ImportError as e:
    print(f" LỖI KHÔNG TÌM THẤY MODULE: {e}")  

data_dir = PROJECT_ROOT / "data"

path_input = data_dir / "diem_chuan_all.csv"
path_output = data_dir / "diem_chuan_chuan_hoa.csv"
path_removed = data_dir / "diem_chuan_bi_loai.csv"

print(path_input)          # để bạn nhìn lại path
print(path_input.exists()) # check có file không (True là ok)

df_final = process_diem_chuan(path_input, path_output, path_removed)